In [1]:
import pandas as pd

import numpy as np

import warnings

from numba.core.errors import NumbaDeprecationWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=NumbaDeprecationWarning)

In [2]:
df = pd.read_csv('data/california_housing/export_epita.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df2 = pd.read_csv('data/california_housing/cities.csv')
#nombre de maisons à prendre en compte (20k c'est beaucoup trop)
# quelques condition pour nettoyer la base de données
df = df.loc[df['Population']<10000]
df = df.loc[df['AveOccup']<6]
df = df.loc[df['AveBedrms']<1.5]
df = df.loc[df['HouseAge']<50]

# Seulement San Francisco :
# Latitude > 37,2
# Latitude < 38,07
# Longitude > -122,5
# Longitude < -121,75

#df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
#df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

save_df = df.copy()

N=1000
n = N/len(df)
X_all = df.iloc[:,0:8].copy()
df1 = df.sample(frac=n, random_state=14)
df1 = df1.reset_index(drop=True)
save_df1 = df1.copy()

X = df1.iloc[:,0:8]
Y = df1.iloc[:,9]

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state = 9)
model.fit(X, Y)

SHAP = df1.iloc[:,[10,11,12,13,14,15,16,17]]

**Démarrage d'AntakIA**

In [4]:
import antakia
import sklearn

mod = sklearn.linear_model.LinearRegression()

explain = antakia.Xplainer(X = X, Y = Y, model = model)

reg1 = antakia.create_save(np.zeros(500), "Sauvegarde importée 1")
reg2 = antakia.create_save(np.concatenate([np.zeros(300), np.ones(200)]), "Sauvegarde importée 2", [mod, mod])
#reg3 = antakia.load_save(local='data_save/ma_sauvegarde.json')[-1]

regions=[reg1, reg2]
#regions=[reg1, reg2, reg3]

display(explain.interface(exp_val=SHAP, default_projection = "PaCMAP", map=True))

Layout(children=[Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\t2\x00\x00\n\xd5\x08\x06\x00\x00\x0…

In [5]:
regles = explain.result()[0]['rules']
regles

TypeError: string indices must be integers

In [ ]:
new_df = antakia.from_rules(X, regles)
new_df